In [1]:
!pip install transformers==4.15.0 sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 5.7 MB/s 
     |████████████████████████████████| 1.2 MB 36.9 MB/s 
     |████████████████████████████████| 101 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 36.5 MB/s 
     |████████████████████████████████| 880 kB 34.2 MB/s 
     |████████████████████████████████| 3.3 MB 45.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3a64598cb9bcaa478b723f8619ceec5c648cd19c2c44704d8995de2c336c43dc
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', model_max_length=512)
tokenizer = pickle.load(open('drive/MyDrive/AIBuilders/json/tokenizer_json_15k.pkl', 'rb'))

In [4]:
ds_tag = pickle.load(open('drive/MyDrive/AIBuilders/json/ner_json_15k_2.pkl', 'rb'))
ds_mlm = pickle.load(open('drive/MyDrive/AIBuilders/json/mlm_json_15k_2.pkl', 'rb'))

In [5]:
df_mlm = ds_mlm[:10576]
df_tag = ds_tag[:10576]

df_tag_train, df_tag_val = np.split(df_tag.sample(frac=1, random_state=42),
                            [int(.9 * len(df_tag))])

df_mlm_train, df_mlm_val = np.split(df_mlm.sample(frac=1, random_state=42),
                            [int(.9 * len(df_mlm))])

NUM_SAMPLE = df_tag_train.shape[0]
NUM_SAMPLE

9518

In [6]:
maxlen = 0
for i in range(9518):
  a = df_tag_train.iloc[i]['text']['input_ids'][0].tolist()
  a = [k for k in a if k != 1]
  # print(a)
  if len(a) > maxlen:
    maxlen = len(a)

print(maxlen)

80


In [7]:
len(tokenizer)

25354

In [8]:
char_vocab = {' ': 2, '!': 5, '"': 25, '#': 4, '$': 3, '%': 6, '&': 7, "'": 24, '(': 26, ')': 27, '*': 8, '+': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 66, '1': 205, '2': 154, '3': 265, '4': 200, '5': 151, '6': 275, '7': 162, '8': 72, '9': 354, ':': 14, ';': 15, '<': 30, '=': 21, '>': 31, '?': 16, '@': 17, '[': 32, ']': 33, '^': 18, '_': 19, '`': 20, 'a': 196, 'b': 194, 'c': 339, 'd': 355, 'e': 300, 'f': 350, 'g': 19, 'h': 83, 'i': 227, 'j': 248, 'k': 144, 'l': 199, 'm': 284, 'n': 234, 'o': 44, 'p': 228, 'pad': 0, 'q': 360, 'r': 34, 's': 270, 't': 169, 'u': 40, 'unknown': 1, 'v': 215, 'w': 195, 'x': 26, 'y': 101, 'z': 78, '{': 28, '|': 22, '}': 29, '~': 23, '\x93': 246, '\x94': 263, '\xa0': 59, '¡': 190, '£': 61, '§': 38, '«': 70, '\xad': 45, '®': 99, '°': 347, '±': 22, '²': 150, '³': 288, '´': 329, 'µ': 206, '·': 173, '¹': 90, '»': 326, '¼': 28, '½': 41, '¾': 239, '×': 112, 'ß': 315, 'à': 43, 'á': 266, 'â': 91, 'ã': 25, 'ä': 289, 'å': 54, 'æ': 211, 'ç': 24, 'è': 338, 'é': 340, 'ê': 21, 'ë': 6, 'ì': 133, 'í': 108, 'î': 223, 'ï': 37, 'ð': 258, 'ñ': 175, 'ò': 9, 'ó': 132, 'ô': 212, 'ö': 60, '÷': 136, 'ø': 302, 'ù': 229, 'ú': 179, 'û': 249, 'ü': 296, 'ý': 75, 'þ': 216, 'ā': 264, 'ă': 349, 'ą': 322, 'č': 279, 'đ': 277, 'ē': 183, 'ę': 335, 'ě': 317, 'ğ': 186, 'ī': 276, 'ı': 318, 'ł': 139, 'ń': 74, 'ŋ': 257, 'ō': 122, 'ŏ': 107, 'ő': 180, 'œ': 67, 'ś': 123, 'ş': 224, 'š': 97, 'ū': 79, 'ŭ': 226, 'ž': 182, 'ơ': 64, 'ư': 76, 'ǎ': 98, 'ǐ': 159, 'ǒ': 198, 'ǔ': 233, 'ɔ': 331, 'ə': 116, 'ɛ': 104, 'ɪ': 165, 'ʃ': 368, 'ʒ': 102, 'ʰ': 220, 'ʿ': 312, 'ˈ': 362, 'ː': 63, '̇': 307, '̍': 292, '̤': 321, '̪': 168, 'α': 303, 'β': 359, 'γ': 236, 'δ': 311, 'ε': 305, 'θ': 304, 'κ': 115, 'λ': 346, 'μ': 27, 'π': 52, 'σ': 313, 'φ': 138, 'χ': 343, 'ψ': 185, 'ω': 278, 'в': 85, 'г': 176, 'ж': 157, 'и': 55, 'й': 214, 'к': 17, 'с': 32, 'ч': 333, 'ш': 46, 'щ': 314, 'ы': 310, 'ь': 92, 'э': 256, 'я': 147, 'ё': 71, 'ө': 204, 'إ': 282, 'ا': 218, 'ب': 201, 'ج': 20, 'ع': 251, 'ف': 301, 'ق': 50, 'ل': 155, 'م': 323, 'ن': 334, 'ه': 110, 'و': 164, 'ي': 178, 'َ': 352, 'ُ': 130, 'ِ': 31, 'ّ': 117, 'ْ': 273, 'द': 89, 'न': 262, 'म': 113, 'व': 268, 'ी': 267, 'े': 306, 'ॐ': 351, 'ก': 342, 'ข': 77, 'ฃ': 36, 'ค': 191, 'ฅ': 167, 'ฆ': 118, 'ง': 163, 'จ': 125, 'ฉ': 260, 'ช': 344, 'ซ': 166, 'ฌ': 353, 'ญ': 308, 'ฎ': 131, 'ฏ': 53, 'ฐ': 135, 'ฑ': 187, 'ฒ': 259, 'ณ': 327, 'ด': 121, 'ต': 126, 'ถ': 15, 'ท': 95, 'ธ': 156, 'น': 73, 'บ': 16, 'ป': 189, 'ผ': 369, 'ฝ': 7, 'พ': 49, 'ฟ': 103, 'ภ': 160, 'ม': 293, 'ย': 243, 'ร': 11, 'ฤ': 320, 'ล': 366, 'ฦ': 325, 'ว': 119, 'ศ': 23, 'ษ': 106, 'ส': 177, 'ห': 274, 'ฬ': 153, 'อ': 235, 'ฮ': 294, 'ฯ': 29, 'ะ': 170, 'ั': 82, 'า': 88, 'ำ': 127, 'ิ': 62, 'ี': 272, 'ึ': 287, 'ื': 111, 'ุ': 5, 'ู': 184, 'ฺ': 161, 'เ': 330, 'แ': 309, 'โ': 337, 'ใ': 232, 'ไ': 210, 'ๅ': 356, 'ๆ': 319, '็': 363, '่': 269, '้': 247, '๊': 158, '๋': 152, '์': 142, 'ํ': 230, '๐': 120, '๑': 143, '๒': 124, '๓': 345, '๔': 8, '๕': 213, '๖': 221, '๗': 13, '๘': 33, '๙': 357, 'ḍ': 271, 'ḥ': 39, 'ṃ': 324, 'ṅ': 255, 'ṇ': 69, 'ṛ': 56, 'ṣ': 286, 'ṭ': 238, 'ạ': 244, 'ả': 250, 'ầ': 252, 'ế': 51, 'ễ': 134, 'ệ': 93, 'ị': 129, 'ọ': 219, 'ộ': 100, 'ỽ': 47, '\u2003': 140, '\u2005': 58, '\u200b': 348, '\u200c': 193, '\u200d': 192, '\u200e': 149, '–': 171, '—': 109, '‘': 336, '’': 358, '“': 87, '”': 137, '†': 128, '‡': 48, '•': 328, '…': 12, '′': 316, '‹': 30, '›': 141, '※': 181, '⁄': 285, '€': 80, '℃': 237, '™': 241, '→': 146, '↔': 209, '∈': 81, '∑': 65, '−': 283, '∘': 172, '√': 96, '∞': 68, '∨': 298, '≈': 10, '≠': 105, '≡': 242, '≤': 197, '≥': 84, '⊕': 14, '⊙': 299, '⋯': 35, '▪': 57, '○': 148, '●': 245, '◦': 225, '☆': 364, '♠': 261, '♣': 295, '♥': 42, '♦': 281, '♪': 18, '♯': 94, '\u3000': 254, '。': 86, '《': 174, '》': 290, '【': 253, 'い': 332, 'う': 217, 'え': 280, 'か': 188, '・': 341, '叉': 114, '夜': 297, '奖': 367, '犬': 240, '获': 365, '虫': 207, '\uf022': 361, '\uf712': 203, '\ufeff': 202, '（': 231, '）': 222, '：': 208, '～': 145, '\U00100083': 291}

In [9]:
WORD_EMBEDDING_DIM = 64
CHAR_EMBEDDING_DIM = 128
# MAX_WORD_IN_SENT = 512
MAX_WORD_IN_SENT = 80
MAX_CHAR_IN_WORD = 30
len(char_vocab)

399

In [10]:
class DetectorModeler(nn.Module):

    def __init__(self, word_vocab_size, word_embedding_dim, max_word_in_sent, char_vocab_size, char_embedding_dim, max_char_in_word):
        super(DetectorModeler, self).__init__()
        self.char_embeddings = nn.Embedding(char_vocab_size, char_embedding_dim)
        self.char_lstm = nn.LSTM(char_embedding_dim * max_word_in_sent * max_char_in_word, hidden_size=32,
                            num_layers=1, bidirectional=True)
        self.word_embeddings = nn.Embedding(word_vocab_size, word_embedding_dim)

        self.lstm = nn.LSTM((word_embedding_dim * max_word_in_sent) + (64), hidden_size=64,
                            num_layers=2, bidirectional=True)
        self.linear = nn.Linear(128, max_word_in_sent)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, word_in, char_in):
        char_embeds = self.char_embeddings(char_in).view((1, -1))
        char_enc, (hidden, cell) = self.char_lstm(char_embeds)
        word_embeds = self.word_embeddings(word_in).view((1, -1))
        # print(char_enc.size())
        # print(word_embeds.size())
        all_embeds = torch.cat((char_enc, word_embeds), 1)
        # print(all_embeds.size())
        out, (hidden, cell) = self.lstm(all_embeds)
        out = self.linear(out)
        # out = F.relu(self.linear1(embeds))
        probs = self.softmax(out)
        # print(probs)
        return probs

In [11]:
losses = []
loss_function = nn.CrossEntropyLoss()
model = DetectorModeler(len(tokenizer), WORD_EMBEDDING_DIM, MAX_WORD_IN_SENT, len(char_vocab), CHAR_EMBEDDING_DIM, MAX_CHAR_IN_WORD)
optimizer = optim.Adam(model.parameters(), lr=0.002)

In [12]:
for epoch in tqdm(range(60)):
    total_loss = 0
    # for sent_id in range(NUM_SAMPLE):
    for sent_id in range(10):
      text = df_tag_train.iloc[sent_id]['text']['input_ids'][0].tolist()
      text = text[:80]
      # print(text)
      # text = [k for k in text if k != 1]
      labels = df_tag_train.iloc[sent_id]['labels'].tolist()
      labels = labels[:80]
      # labels = [k for k in labels if k != 1]
      labels_len = len(labels)
      for i in range(labels_len):
        if labels[i] == -100:
          labels[i] = 0

      sent_len = len(text)
      char_in = []
      for i in range(sent_len):
        word = tokenizer.convert_ids_to_tokens(text[i])
        word = word.replace("▁", " ")
        # print(f"WORD: {word}")
        char_list = [char_vocab[j] for j in word]
        while len(char_list) < 30:
          char_list.append(0)
        # print(char_list)
        char_in.append(char_list)

      char_in = torch.tensor(char_in, dtype=torch.long)
      word_in = torch.tensor([text], dtype=torch.long)
      # print(char_in.size())
      # print(word_in.size())
      model.zero_grad()

      probs = model(word_in, char_in)
      # print(f"SENT PROBS SHAPE: {probs.shape}")
      # print(probs[0][1])
      labels = torch.tensor([labels], dtype=torch.float)
      # print(labels.shape)

      loss = loss_function(probs, labels)

      # for i in range(512):
      #   probs_val = probs[0][i].item()
      #   labels_val = labels[0][i].item()
        # print(probs_val, labels_val)        

      loss.backward()
      optimizer.step()

      total_loss += loss.item()
      print(f"SENT #{sent_id} LOSS: {loss.item()}")
    print(f"EPOCH LOSS: {total_loss}")
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

  0%|          | 0/60 [00:00<?, ?it/s]

SENT #0 LOSS: 8.762606620788574
SENT #1 LOSS: 13.146073341369629
SENT #2 LOSS: 8.764236450195312
SENT #3 LOSS: 17.525470733642578
SENT #4 LOSS: 13.129606246948242
SENT #5 LOSS: 4.371227264404297
SENT #6 LOSS: 4.376549243927002
SENT #7 LOSS: 4.384583473205566
SENT #8 LOSS: 13.115234375


  2%|▏         | 1/60 [00:18<18:01, 18.33s/it]

SENT #9 LOSS: 4.381288528442383
EPOCH LOSS: 91.95687627792358
SENT #0 LOSS: 8.718242645263672
SENT #1 LOSS: 13.103628158569336
SENT #2 LOSS: 8.750852584838867
SENT #3 LOSS: 17.48345184326172
SENT #4 LOSS: 13.034503936767578
SENT #5 LOSS: 4.312526702880859
SENT #6 LOSS: 4.353393077850342
SENT #7 LOSS: 4.382894992828369
SENT #8 LOSS: 12.952598571777344


  3%|▎         | 2/60 [00:35<17:20, 17.93s/it]

...

SENT #9 LOSS: 4.403239727020264
EPOCH LOSS: 88.4696261882782
SENT #0 LOSS: 7.8070173263549805
SENT #1 LOSS: 13.209354400634766
SENT #2 LOSS: 8.806473731994629
SENT #3 LOSS: 17.61266326904297
SENT #4 LOSS: 12.209983825683594
SENT #5 LOSS: 3.4041683673858643
SENT #6 LOSS: 4.403199195861816
SENT #7 LOSS: 4.403255462646484
SENT #8 LOSS: 12.210247993469238


 98%|█████████▊| 59/60 [18:10<00:19, 19.91s/it]

SENT #9 LOSS: 4.40324068069458
EPOCH LOSS: 88.46960425376892
SENT #0 LOSS: 7.807004451751709
SENT #1 LOSS: 13.209367752075195
SENT #2 LOSS: 8.806476593017578
SENT #3 LOSS: 17.612674713134766
SENT #4 LOSS: 12.209978103637695
SENT #5 LOSS: 3.4041426181793213
SENT #6 LOSS: 4.403201580047607
SENT #7 LOSS: 4.403255939483643
SENT #8 LOSS: 12.210236549377441


100%|██████████| 60/60 [18:30<00:00, 18.51s/it]

SENT #9 LOSS: 4.403242111206055
EPOCH LOSS: 88.46958041191101
[91.95687627792358, 91.46515417098999, 90.07308435440063, 88.9854691028595, 88.63567900657654, 88.53703880310059, 88.50786447525024, 88.49545812606812, 88.48909187316895, 88.48520636558533, 88.48253893852234, 88.4805417060852, 88.4789788722992, 88.47772526741028, 88.4767005443573, 88.47585654258728, 88.4751489162445, 88.47455430030823, 88.47404766082764, 88.47360754013062, 88.47322821617126, 88.47289323806763, 88.47260093688965, 88.47233510017395, 88.47209978103638, 88.47188782691956, 88.47169542312622, 88.47151803970337, 88.47136211395264, 88.47131991386414, 88.47131562232971, 88.47117352485657, 88.47103953361511, 88.47091579437256, 88.47080397605896, 88.47070240974426, 88.4706060886383, 88.47052192687988, 88.47044324874878, 88.47036719322205, 88.47030210494995, 88.47023916244507, 88.47018027305603, 88.47012448310852, 88.47007369995117, 88.47002696990967, 88.46998023986816, 88.46994209289551, 88.46989703178406, 88.469862699